In [21]:
import os
import torch 
import argparse
import yaml
import wandb
from box import Box
import pandas as pd 

from data.data_processing import DataProcessor
from models.models import LSTMModel,CNN_LSTMModel
from utils.utils import train, model_eval_metric,test


with open('config.yaml', 'r') as f:
    config = Box(yaml.safe_load(f))

In [10]:

# Data Processing
data_processor = DataProcessor(config)
data_processor.load_data()
data_processor.preprocess_data()


/home/robotics20/roboticstau/FMG_project/data/data_processing.py:59: FutureWarning: The 'axis' keyword in DataFrame.rolling is deprecated and will be removed in a future version. Call the method without the axis keyword instead.
  self.data[self.config.fmg_index] = self.data[self.config.fmg_index].rolling(window=self.config.window_size, axis=0).mean()
/home/robotics20/roboticstau/FMG_project/data/data_processing.py:60: FutureWarning: The 'axis' keyword in DataFrame.rolling is deprecated and will be removed in a future version. Call the method without the axis keyword instead.
  self.data[self.labal_index] = self.data[self.labal_index].rolling(window=self.config.window_size, axis=0).mean()


In [11]:
data = data_processor.data

In [15]:
labels = data[config.first_positoin_label_inedx]

In [16]:
labels

,M1x,M1y,M1z,M2x,M2y,M2z,M3x,M3y,M3z,M4x,M4y,M4z
0,1.656635,0.411705,0.548710,-0.201934,-0.355287,0.196353,0.106074,-0.786448,0.164558,0.094593,-0.855975,-0.121607
1,1.656633,0.411681,0.548725,-0.201833,-0.355115,0.196389,0.106097,-0.786419,0.164576,0.094576,-0.855958,-0.121583
2,1.656633,0.411681,0.548725,-0.201733,-0.354943,0.196424,0.106121,-0.786390,0.164594,0.094559,-0.855941,-0.121558
3,1.656633,0.411660,0.548737,-0.201632,-0.354764,0.196455,0.106142,-0.786365,0.164609,0.094542,-0.855925,-0.121538
4,1.656633,0.411660,0.548737,-0.201531,-0.354586,0.196485,0.106162,-0.786340,0.164624,0.094525,-0.855909,-0.121517
...,...,...,...,...,...,...,...,...,...,...,...,...
99954,1.679291,0.399659,0.636447,-0.437005,-0.530871,0.653844,-0.636368,0.288710,-0.401825,-0.269678,0.571457,-0.412351
99955,1.679234,0.399331,0.636433,-0.437698,-0.531162,0.651326,-0.635306,0.276080,-0.402656,-0.272220,0.560093,-0.420182
99956,1.679178,0.399152,0.636110,-0.438842,-0.531596,0.648647,-0.634217,0.262832,-0.403604,-0.274915,0.548145,-0.428258
99957,1.679021,0.398974,0.636165,-0.440553,-0.532173,0.645548,-0.633066,0.248951,-0.404795,-0.277769,0.535595,-0.436560


In [17]:
import numpy as np
import numpy as np

def calculate_vectors(data):
    M1 = data[['M1x', 'M1y', 'M1z']].values
    M2 = data[['M2x', 'M2y', 'M2z']].values
    M3 = data[['M3x', 'M3y', 'M3z']].values
    M4 = data[['M4x', 'M4y', 'M4z']].values
    V_M2M1 = M1 - M2
    V_M1M4 = M4 - M1
    V_M4M3 = M3 - M4
    return V_M2M1, V_M1M4, V_M4M3

def calculate_angles_with_axes(vectors, unit_vectors):



    angles = {axis: [] for axis in unit_vectors}  # Initialize as a dictionary of lists

    for vector in vectors:
        vector = np.array(vector)
        vector_magnitude = np.linalg.norm(vector)

        # Check for zero magnitude to avoid division by zero
        if vector_magnitude == 0:
            for axis in unit_vectors:
                angles[axis].append(0)
            continue

        for axis, unit_vector in unit_vectors.items():
            dot_product = np.dot(vector, unit_vector)
            # Clamp the value to the valid range for arccos to avoid numerical errors
            cos_angle = np.clip(dot_product / vector_magnitude, -1, 1)
            angle_rad = np.arccos(cos_angle)
            angles[axis].append(np.degrees(angle_rad))
    
    return angles

# Calculating vectors
V_M2M1, V_M1M4, V_M4M3 = calculate_vectors(data)

# Unit vectors for the reference frame
unit_vectors = {
    'X-axis': np.array([1, 0, 0]),
    'Y-axis': np.array([0, 1, 0]),
    'Z-axis': np.array([0, 0, 1])
}

In [19]:
angles = calculate_angles_with_axes(V_M1M4, unit_vectors)



In [22]:
angles_df = pd.DataFrame(angles)


In [32]:
angles_df["X-axis"] = np.radians(angles_df["X-axis"])
angles_df["Y-axis"] = np.radians(angles_df["Y-axis"])
angles_df["Z-axis"] = np.radians(angles_df["Z-axis"])

In [33]:
angles_df.to_csv(r'angles.csv')